#Task-4

###(A) Formulate And Solve A Business Optimization Problem Using Cost Minimization.

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 46.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, value

# Load dataset
df = pd.read_csv('/content/supply_chain_dataset.csv')

# Aggregate counts
supply = df['Warehouse_block'].value_counts().to_dict()
demand = df['Mode_of_Shipment'].value_counts().to_dict()

warehouses = list(supply.keys())
shipments = list(demand.keys())

# Simulated costs: average cost + discount for each warehouse-shipment pair
cost_matrix = df.groupby(['Warehouse_block', 'Mode_of_Shipment'])[['Cost_of_the_Product', 'Discount_offered']].mean().sum(axis=1).to_dict()

# Initialize LP model
model = LpProblem("Shipping_Cost_Minimization", LpMinimize)

# Decision variables: number of shipments from warehouse to shipment mode
x = LpVariable.dicts("Route", (warehouses, shipments), lowBound=0, cat='Continuous')

# Objective: minimize total cost
model += lpSum(cost_matrix.get((w, s), 9999) * x[w][s] for w in warehouses for s in shipments)

# Supply constraint
for w in warehouses:
    model += lpSum(x[w][s] for s in shipments) <= supply[w], f"Supply_{w}"

# Demand constraint
for s in shipments:
    model += lpSum(x[w][s] for w in warehouses) >= demand[s], f"Demand_{s}"

# Solve model
model.solve()
print(f"Status: {LpStatus[model.status]}")
print("Optimal shipments:")
for w in warehouses:
    for s in shipments:
        shipped = x[w][s].varValue
        if shipped > 0:
            print(f" {w} → {s}: {shipped:.2f} units")

print(f"Total Estimated Cost: ₹{value(model.objective):,.2f}")


Status: Optimal
Optimal shipments:
 F → Ship: 1889.00 units
 F → Flight: 1777.00 units
 D → Ship: 74.00 units
 D → Road: 1760.00 units
 A → Ship: 1833.00 units
 B → Ship: 1833.00 units
 C → Ship: 1833.00 units
Total Estimated Cost: ₹2,452,387.77


###(B) Formulate And Solve A Business Optimization Problem Using Resource Allocation.

In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value

# Load the dataset
df = pd.read_csv('/content/supply_chain_dataset.csv')

# Aggregate average customer rating (as profit), and average cost (as resource usage)
summary = df.groupby('Mode_of_Shipment').agg({
    'Customer_rating': 'mean',
    'Cost_of_the_Product': 'mean'
}).reset_index()

# Set values for resource allocation
profit = summary.set_index('Mode_of_Shipment')['Customer_rating'].to_dict()
cost = summary.set_index('Mode_of_Shipment')['Cost_of_the_Product'].to_dict()
modes = list(profit.keys())

# Total shipping budget available
budget = 100000  # you can change this value

# Initialize model
model = LpProblem("Resource_Allocation_Maximize_Satisfaction", LpMaximize)

# Decision variables: how many units to allocate per shipping mode
x = LpVariable.dicts("Allocate", modes, lowBound=0, cat='Continuous')

# Objective: maximize total satisfaction
model += lpSum(profit[m] * x[m] for m in modes)

# Constraint: total cost must not exceed budget
model += lpSum(cost[m] * x[m] for m in modes) <= budget, "BudgetConstraint"

# Solve
model.solve()

# Output results
print(f"Status: {LpStatus[model.status]}")
print("Optimal Resource Allocation:")
for m in modes:
    print(f"  {m}: {x[m].varValue:.2f} units")

print(f"Total Customer Satisfaction Score: {value(model.objective):.2f}")


Status: Optimal
Optimal Resource Allocation:
  Flight: 477.77 units
  Road: 0.00 units
  Ship: 0.00 units
Total Customer Satisfaction Score: 1424.97
